In [1]:
import os
from pathlib import Path

from src import dino


# Output directory (_output is ignored by git)
OUTPUT_DIR = "_output"
if not os.path.isdir(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    print("Created output directory:", OUTPUT_DIR)


IMAGE_DIR = Path("images")
def file_to_brightness(file_path, model, white_nits=200):
    image = dino.read_image(file_path)
    L = dino.rgb_to_relative_luminance(image)
    L *= white_nits
    return model(L)

In [3]:
from src import plotting

## Apply model to all illusions
ILLUSION_IMAGES = [
    "mondrian1.png",
    "sunflower.png",
    "noisy_contrast.png",
]

for image in ILLUSION_IMAGES:
    for name, model in [
        ("dinos", dino.dn_brightness_model),
        ("blakeslee97", dino.blakeslee97_brightness_model),
        ("blommaert", dino.blommaert_brightness_model),
    ]:
        brightness = file_to_brightness(str(IMAGE_DIR / image), model)
        filename = f"{os.path.splitext(image)[0]}_{name}.html"
        filepath = os.path.join(OUTPUT_DIR, filename)
        plotting.image_height_plot(
            brightness,
            output_path=filepath,
            display=False,
        )
        print("Exported", filepath)

Exported _output/mondrian1_dinos.html
Exported _output/mondrian1_blakeslee97.html
Exported _output/mondrian1_blommaert.html
Exported _output/sunflower_dinos.html
Exported _output/sunflower_blakeslee97.html
Exported _output/sunflower_blommaert.html
Exported _output/noisy_contrast_dinos.html
Exported _output/noisy_contrast_blakeslee97.html
Exported _output/noisy_contrast_blommaert.html


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.colors import Normalize
from tqdm.auto import tqdm


IMAGE_POINTS_OF_INTEREST = {
    "ramp_squares.png": [
        (470, 330), (850, 330),
        
    ],
    "ramp_bar.png": [
        (420, 330), (880, 330),
    ],
    "whites.png": [
        (570, 410), (1350, 540),
    ],
}
MODELS = [
    dino.blommaert_brightness_model,
    dino.blakeslee97_brightness_model,
    dino.dn_brightness_model,
]

for image, pois in tqdm(IMAGE_POINTS_OF_INTEREST.items()):
    image_path = str(IMAGE_DIR / image)
    fig, axs = plt.subplots(1+len(MODELS), 1)
    upper = axs[0]
    original_img = mpimg.imread(image_path)
    upper.imshow(original_img, cmap="Greys", aspect="equal")
    brightness = file_to_brightness(image_path)
    norm = Normalize(vmin=-0.1, vmax=0.2, clip=True)
    im = lower.imshow(brightness, cmap="viridis", norm=norm, aspect="equal")
    fig.colorbar(im, ax=axs.ravel().tolist())
    for px, py in pois:
        lower.plot(px, py, marker=".", color="red", markersize=2)
        lower.text(
            px, py-80,
            str(round(brightness[py,px], 3)),
            color="red",
            fontsize=10,
            horizontalalignment="center",
        )
    for ax in axs:
        ax.set_xticks([])
        ax.set_xticklabels([])
        ax.set_yticks([])
        ax.set_yticklabels([])
    # file_path = Path("_output") / f"{(IMAGE_DIR / image).stem}.pdf"
    # fig.savefig(file_path, bbox_inches="tight", dpi=300, transparent=True)
    # print(f"Exported {file_path}")